In [28]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px


df_main_categ = pd.read_csv('df_main_categ.csv')

mms = MinMaxScaler()
selected_cols = ['price_sqm', 'floor_area_sqm','year','distance_to_nearest_mrt','distance_to_downtown','storey_avg','toilet_number',
       'room_number']

selected_cols_without_year = ['price_sqm', 'floor_area_sqm','distance_to_nearest_mrt','distance_to_downtown','storey_avg','toilet_number',
       'room_number']

df_by_year = df_main_categ[selected_cols].groupby('year').mean()
df_by_year[selected_cols_without_year] = mms.fit_transform(df_by_year[selected_cols_without_year])
df_by_year = df_by_year.reset_index()
df_by_year.columns

In [35]:
df_by_year.columns

Index(['year', 'price_sqm', 'floor_area_sqm', 'distance_to_nearest_mrt',
       'distance_to_downtown', 'storey_avg', 'toilet_number', 'room_number'],
      dtype='object')

In [74]:
fig = px.line(df_by_year, x="year", y=selected_cols_without_year)
# fig.show()

In [80]:
df_main_categ

,Unnamed: 0,year,floor_area_sqm,remaining_lease,price_sqm,distance_to_downtown,distance_to_nearest_mrt,toilet_number,room_number,town_ANG MO KIO,...,storey < 10,storey < 15,storey < 20,storey < 30,story > 30,Bus,Entertainment,Food,School,Shop
0,0,2017,44.0,61,5272.727273,8.383132,0.851339,1,2,1,...,0,1,0,0,0,1.0,0.0,11.0,0.0,4.0
1,1,2017,67.0,60,3731.343284,9.236619,0.696968,2,3,1,...,1,0,0,0,0,2.0,0.0,7.0,0.0,0.0
2,2,2017,67.0,62,3910.447761,8.911143,0.617914,2,3,1,...,1,0,0,0,0,0.0,1.0,3.0,0.0,1.0
3,3,2017,68.0,62,3897.058824,8.383132,0.851339,2,3,1,...,1,0,0,0,0,1.0,0.0,11.0,0.0,4.0
4,4,2017,67.0,62,3955.223881,8.911143,0.617914,2,3,1,...,1,0,0,0,0,0.0,1.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839605,839605,1999,142.0,69,3211.267606,13.132845,0.637828,2,3,0,...,0,1,0,0,0,0.0,1.0,9.0,0.0,1.0
839606,839606,1999,142.0,70,2873.239437,14.026676,0.411571,2,3,0,...,1,0,0,0,0,0.0,2.0,25.0,0.0,8.0
839607,839607,1999,146.0,70,3212.328767,14.230671,1.670481,2,3,0,...,1,0,0,0,0,0.0,1.0,3.0,0.0,0.0
839608,839608,1999,146.0,70,3013.698630,13.932028,1.254211,2,3,0,...,1,0,0,0,0,0.0,2.0,7.0,0.0,2.0


#### rank the variance of the variable

In [103]:
df_var = df_main_categ.var().to_frame()
df_var.columns = ['vars']
df_var.sort_values(by=['vars'])

,vars
flat_model_2-room,5.002078e-05
town_LIM CHU KANG,7.622015e-05
flat_model_Type S2,1.679071e-04
flat_model_Type S1,3.333773e-04
flat_type_MULTI-GENERATION,6.034879e-04
...,...
floor_area_sqm,6.779063e+02
distance_to_nearest_mrt,2.106709e+05
distance_to_downtown,2.108512e+05
price_sqm,1.661236e+06


#### However one-hot still useful

In [73]:
df_main_categ = pd.read_csv('df_main_categ.csv')

mms = MinMaxScaler()
selected_cols = ['price_sqm', 'floor_area_sqm','distance_to_nearest_mrt','distance_to_downtown','storey_avg','toilet_number',
       'room_number']

selected_cols_without_storey = ['price_sqm', 'floor_area_sqm','distance_to_nearest_mrt','distance_to_downtown','toilet_number',
       'room_number']

df_by_storey = df_main_categ[selected_cols].groupby('storey_avg').mean()
df_by_storey[selected_cols_without_storey] = mms.fit_transform(df_by_storey[selected_cols_without_storey])
df_by_storey = df_by_storey.reset_index()
fig = px.line(df_by_storey, x="storey_avg", y=selected_cols_without_storey)
# fig.show()

In [65]:
df_non_hot_categ = pd.read_csv('df_non_hot.csv')
df_non_hot_categ = df_non_hot_categ[['year',
       'floor_area_sqm', 'resale_price', 
       'remaining_lease', 
       'price_sqm', 'distance_to_downtown',
       'distance_to_nearest_mrt', 'toilet_number', 'room_number', 'Bus',
       'Entertainment', 'Food', 'School', 'Shop']]

In [66]:
df_non_hot_categ

,year,floor_area_sqm,resale_price,remaining_lease,price_sqm,distance_to_downtown,distance_to_nearest_mrt,toilet_number,room_number,Bus,Entertainment,Food,School,Shop
0,2017,44.0,232000.0,61,5272.727273,8.383132,0.851339,1,2,1.0,0.0,11.0,0.0,4.0
1,2017,67.0,250000.0,60,3731.343284,9.236619,0.696968,2,3,2.0,0.0,7.0,0.0,0.0
2,2017,67.0,262000.0,62,3910.447761,8.911143,0.617914,2,3,0.0,1.0,3.0,0.0,1.0
3,2017,68.0,265000.0,62,3897.058824,8.383132,0.851339,2,3,1.0,0.0,11.0,0.0,4.0
4,2017,67.0,265000.0,62,3955.223881,8.911143,0.617914,2,3,0.0,1.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839605,1999,142.0,456000.0,69,3211.267606,13.132845,0.637828,2,3,0.0,1.0,9.0,0.0,1.0
839606,1999,142.0,408000.0,70,2873.239437,14.026676,0.411571,2,3,0.0,2.0,25.0,0.0,8.0
839607,1999,146.0,469000.0,70,3212.328767,14.230671,1.670481,2,3,0.0,1.0,3.0,0.0,0.0
839608,1999,146.0,440000.0,70,3013.698630,13.932028,1.254211,2,3,0.0,2.0,7.0,0.0,2.0


In [76]:
fig = px.scatter_matrix(df_non_hot_categ,
    dimensions=['price_sqm', 'distance_to_downtown',
       'distance_to_nearest_mrt', 'toilet_number', 'room_number', 'Bus',
       'Entertainment', 'Food', 'School', 'Shop'],
    color="year", width=1000, height=1000)
# fig.show()